# How to Implement Prompt Chaining
Implementing prompt chaining involves a systematic approach to breaking down a complex task and guiding an LLM through a series of well-defined steps. 

### Step 1: Setting Up the Environment
First, we need to import the required libraries. We'll use the `OpenAI` class from the `openai` package and `os` for handling environment variables.

In [59]:
import os
from openai import OpenAI

Storing sensitive information, like API keys, securely is crucial. One way to achieve this is by setting the API key as an environment variable. You can do it by using an OpenAI API key.

In [46]:
os.environ["OPENAI_API_KEY"] = 'your-api-key-here'

Make sure to replace `'your-api-key-here'` with your actual OpenAI API key!

With the API key set, you can now initialize the OpenAI client. This client will be used to make API calls to OpenAI’s services.


In [ ]:
# Initialize the OpenAI client
client = OpenAI()

### Step 2: Defining a function to interact with OpenAI's chat completions API.
Now, we’ll create a Python function to interact with OpenAI's chat completions API. This function will send a prompt to the API and return the generated response.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

This function takes a prompt and an optional model parameter, makes an API call to OpenAI using the chat completions endpoint, sets up the conversation with a system message and a user message (the prompt), and sets the temperature to 0 for more deterministic outputs. It returns the content of the response or None if an error occurs.

### Step 3: Chaining Multiple Prompts
Now we'll create a Python function that chains together multiple prompts, feeding the output of one prompt as the input to the next.


In [ ]:
def prompt_chain(initial_prompt, follow_up_prompts):
    result = get_completion(initial_prompt)
    if result is None:
        return "Initial prompt failed."

    print(f"Initial output: {result}\n")

    for i, prompt in enumerate(follow_up_prompts, 1):
        full_prompt = f"{prompt}\n\nPrevious output: {result}"
        result = get_completion(full_prompt)
        if result is None:
            return f"Prompt {i} failed."
        print(f"Step {i} output: {result}\n")

    return result

This function implements prompt chaining. It starts with an initial prompt and gets its completion, then iterates through a list of follow-up prompts. For each follow-up prompt, it combines the prompt with the previous output, gets a completion for this combined prompt, and updates the result. If any step fails, it returns an error message. The output of each step is printed for visibility.

### Step 4: Example Usage
We are now going to demonstrate how to use the `prompt_chain` function to create a sequence of prompts that build on each other. This example will focus on summarizing key trends in global temperature changes and exploring related scientific studies and mitigation strategies

In [47]:
# Example usage
initial_prompt = "Summarize the key trends in global temperature changes over the past century."

follow_up_prompts = [
    "Based on the trends identified, list the major scientific studies that discuss the causes of these changes.",
    "Summarize the findings of the listed studies, focusing on the impact of climate change on marine ecosystems.",
    "Propose three strategies to mitigate the impact of climate change on marine ecosystems based on the summarized findings."
]

final_result = prompt_chain(initial_prompt, follow_up_prompts)
print("Final result:", final_result)

Initial output: Over the past century, global temperatures have been steadily increasing due to human activities such as burning fossil fuels and deforestation. This has led to a rise in average global temperatures, resulting in more frequent and severe heatwaves, melting ice caps, rising sea levels, and changes in weather patterns. Scientists have observed that the rate of temperature increase has been accelerating in recent decades, highlighting the urgent need for action to mitigate climate change and its impacts.

Step 1 output: Here are some major scientific studies that discuss the causes of the changes in global temperatures and climate:

1. Intergovernmental Panel on Climate Change (IPCC) Reports: The IPCC regularly assesses the latest scientific research on climate change, including the causes and impacts of global warming. Their reports provide comprehensive overviews of the current state of knowledge on the subject.

2. National Climate Assessment (NCA): The NCA is a US gove


### How It Works Together
1. The initial prompt gets a summary of temperature trends.
2. The first follow-up uses this summary to find relevant studies.
3. The second follow-up summarizes these studies' findings about marine ecosystems.
4. The final prompt uses all this information to propose mitigation strategies.

This chaining allows for a step-by-step approach to complex queries, where each step builds on the information from the previous steps.


## Prompt Chaining Techniques
Prompt chaining can be implemented in various ways to suit different types of tasks and requirements. Here, we explore three primary techniques: Sequential Chaining, Conditional Chaining, and Looping Chaining.

### Sequential Chaining
Sequential chaining involves linking prompts in a straightforward, linear sequence. Each prompt depends on the output of the previous one, creating a step-by-step flow of information and tasks. This technique is ideal for tasks that require a logical progression from one stage to the next such as:
Text Summarization: Breaking down a long document into summarized sections, then combining those summaries into a cohesive overall summary.
Code Generation: Generating code snippets step-by-step, such as first creating function definitions, then implementing those functions, and finally writing test cases.

The code snippet in the previous section is an example of sequential chaining.


### Conditional Chaining
Conditional chaining introduces branching into the prompt chain based on the LLM's output. This technique allows for more flexible and adaptable workflows, enabling the LLM to take different paths depending on the responses it generates.

Let’s see how we can implement conditional chaining to perform sentiment analysis.


In [40]:
def analyze_sentiment(text):
    prompt = f"Analyze the sentiment of the following text and respond with only one word - 'positive', 'negative', or 'neutral': {text}"
    sentiment = get_completion(prompt)
    return sentiment.strip().lower()

def conditional_prompt_chain(initial_prompt):
    result = get_completion(initial_prompt)
    if result is None:
        return "Initial prompt failed."

    print(f"Initial output: {result}\n")

    sentiment = analyze_sentiment(result)
    print(f"Sentiment: {sentiment}\n")

    if sentiment == 'positive':
        follow_up = "Given this positive outlook, what are three potential opportunities we can explore?"
    elif sentiment == 'negative':
        follow_up = "Considering these challenges, what are three possible solutions we can implement?"
    else:  # neutral
        follow_up = "Based on this balanced view, what are three key areas we should focus on for a comprehensive approach?"

    final_result = get_completion(f"{follow_up}\n\nContext: {result}")
    return final_result

# Example usage
initial_prompt = "Analyze the current state of renewable energy adoption globally."

final_result = conditional_prompt_chain(initial_prompt)
print("Final result:", final_result)

Initial output: Renewable energy adoption has been increasing globally in recent years, driven by a combination of factors such as environmental concerns, government policies, technological advancements, and cost competitiveness. Here are some key points to consider when analyzing the current state of renewable energy adoption:

1. **Growing Capacity**: The global capacity of renewable energy sources, such as solar, wind, hydro, and geothermal, has been steadily increasing. According to the International Renewable Energy Agency (IRENA), renewable capacity grew by 176 gigawatts in 2019, reaching a total of 2,537 GW.

2. **Falling Costs**: The costs of renewable energy technologies, particularly solar and wind, have been declining rapidly, making them more competitive with traditional fossil fuels. This cost reduction has been a significant driver of adoption in many countries.

3. **Policy Support**: Many countries have implemented policies and incentives to promote the adoption of rene

First, we define a new function `analyze_sentiment()` that uses the language model to determine the sentiment of a given text. Then, in the `conditional_prompt_chain()` function, we start with the initial prompt about renewable energy adoption. After getting the initial response, we analyze its sentiment. Based on the sentiment, we choose a different follow-up prompt:
 For positive sentiment, we ask about opportunities.
 For negative sentiment, we ask about solutions.
 For neutral sentiment, we ask about key focus areas.

The chosen follow-up prompt is then sent to the language model along with the context from the initial response. The final result is the response to this conditional follow-up prompt.

This approach demonstrates conditional chaining because:
The choice of the second prompt is not predetermined but depends on the content of the first response.
The flow of the conversation adapts based on the sentiment analysis, showcasing how the chain can branch into different paths.
It allows for more dynamic and context-aware interactions with the language model.


### Looping Chaining
Looping chaining involves creating loops within a prompt chain to iterate over data or perform repetitive tasks. This technique is useful when dealing with lists or collections of items that require similar processing steps. Some of the benefits and challenges of this approach are:
* Efficiency: Automates repetitive tasks, saving time and effort.
* Data Processing: Suitable for tasks like processing multiple records, batch summarization, or iterative improvements.
* Challenges: Requires careful handling to avoid infinite loops and ensure that each iteration produces meaningful progress.

Let’s see how looping chaining would be implement for a text completeness task:


In [48]:

def check_completeness(text):
    prompt = f"Analyze the following text and respond with only 'complete' if it covers all necessary aspects, or 'incomplete' if more information is needed:\n\n{text}"
    response = get_completion(prompt)
    return response.strip().lower() == 'complete'

def looping_prompt_chain(initial_prompt, max_iterations=5):
    current_response = get_completion(initial_prompt)
    if current_response is None:
        return "Initial prompt failed."

    print(f"Initial output: {current_response}\n")

    iteration = 0
    while iteration < max_iterations:
        if check_completeness(current_response):
            print(f"Complete response achieved after {iteration + 1} iterations.")
            return current_response

        print(f"Iteration {iteration + 1}: Response incomplete. Expanding...")
        expand_prompt = f"The following response is incomplete. Please expand on it to make it more comprehensive:\n\n{current_response}"
        new_response = get_completion(expand_prompt)
        
        if new_response is None:
            return f"Expansion failed at iteration {iteration + 1}."
        
        current_response = new_response
        print(f"Expanded response: {current_response}\n")
        iteration += 1

    print(f"Maximum iterations ({max_iterations}) reached without achieving completeness.")
    return current_response

# Example usage
initial_prompt = "Explain the process of photosynthesis."

final_result = looping_prompt_chain(initial_prompt)
print("Final result:", final_result)

Initial output: Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy stored in glucose molecules. This process involves several steps:

1. Absorption of light: Chlorophyll, a green pigment found in chloroplasts of plant cells, absorbs light energy from the sun.

2. Water uptake: Plants absorb water from the soil through their roots and transport it to the chloroplasts in the leaves.

3. Splitting of water: In the presence of light energy, water molecules are split into oxygen, protons, and electrons. This process is known as photolysis.

4. Production of ATP and NADPH: The light energy is used to convert ADP and inorganic phosphate into ATP and to reduce NADP+ to NADPH. These molecules are energy carriers that will be used in the next steps of photosynthesis.

5. Carbon dioxide fixation: Carbon dioxide from the air is taken up by the plant through small openings called stomata on the leaves. The e

First, we define a new function `check_completeness()` that uses the language model to determine if a given response is complete or needs more information.
Then, in the `looping_prompt_chain()` function, we start with the initial prompt about photosynthesis. After getting the initial response, we enter a loop:
 We check if the current response is complete using `check_completeness()`.
  If it's complete, we exit the loop and return the result.
  If it's incomplete, we generate a new prompt asking to expand on the previous response.
 We then get a new response based on this expansion prompt.

This process continues until either we get a response deemed complete, or we reach the maximum number of iterations (default is 5).

The loop ensures that we keep refining and expanding the response until it's considered complete or we hit the iteration limit.

This approach demonstrates looping chaining because:
It repeatedly processes and refines the output.
The chain continues in a loop until a specific condition (completeness) is met.
It allows for iterative improvement of the response.
It has a safeguard (max iterations) to prevent infinite loops.


## Practical Applications of Prompt Chaining
Prompt chaining can significantly enhance the capabilities of LLMs across various applications. In this section, we explore some practical uses of prompt chaining, demonstrating how it can be applied to real-world tasks.

### Question Answering over Documents
Prompt chaining can be used to summarize long documents and then generate answers to specific questions. This involves first breaking down the document into manageable sections, summarizing each section, and then using these summaries to answer detailed questions.

Let’s see how to implement this:

In [49]:
def split_document(document, max_length=1000):
    """Split the document into sections of approximately max_length characters."""
    words = document.split()
    sections = []
    current_section = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 > max_length and current_section:
            sections.append(' '.join(current_section))
            current_section = []
            current_length = 0
        current_section.append(word)
        current_length += len(word) + 1

    if current_section:
        sections.append(' '.join(current_section))

    return sections

def summarize_section(section):
    prompt = f"Summarize the following text in a concise manner:\n\n{section}"
    return get_completion(prompt)

def answer_question(summaries, question):
    context = "\n\n".join(summaries)
    prompt = f"Given the following context, answer the question:\n\nContext:\n{context}\n\nQuestion: {question}"
    return get_completion(prompt)

def document_qa(document, questions):
    # Step 1: Split the document
    sections = split_document(document)
    print(f"Document split into {len(sections)} sections.")

    # Step 2: Summarize each section
    summaries = []
    for i, section in enumerate(sections):
        summary = summarize_section(section)
        summaries.append(summary)
        print(f"Section {i+1} summarized.")

    # Step 3: Answer questions
    answers = []
    for question in questions:
        answer = answer_question(summaries, question)
        answers.append((question, answer))

    return answers

# Example usage
long_document = """
[Insert a long document here. For brevity, I'm using a placeholder. 
In a real scenario, this would be a much longer text, perhaps several 
paragraphs or pages about a specific topic.]

This is a long document about climate change. It discusses various aspects 
including causes, effects, and potential solutions. The document covers 
topics such as greenhouse gas emissions, rising global temperatures, 
melting ice caps, sea level rise, extreme weather events, impact on 
biodiversity, and strategies for mitigation and adaptation.

The document also explores the economic implications of climate change, 
international agreements like the Paris Agreement, renewable energy 
technologies, and the role of individual actions in combating climate change.

[Continue with more detailed information about climate change...]
"""

questions = [
    "What are the main causes of climate change mentioned in the document?",
    "What are some of the effects of climate change discussed?",
    "What solutions or strategies are proposed to address climate change?"
]

results = document_qa(long_document, questions)

for question, answer in results:
    print(f"\nQ: {question}")
    print(f"A: {answer}")

Document split into 1 sections.
Section 1 summarized.

Q: What are the main causes of climate change mentioned in the document?
A: The main causes of climate change mentioned in the document include greenhouse gas emissions from human activities such as burning fossil fuels, deforestation, industrial processes, and agriculture. These activities release carbon dioxide, methane, and other greenhouse gases into the atmosphere, leading to the trapping of heat and the warming of the planet.

Q: What are some of the effects of climate change discussed?
A: Some of the effects of climate change discussed in the text include rising global temperatures, melting ice caps and glaciers, sea level rise, extreme weather events such as hurricanes and droughts, shifts in ecosystems and habitats, threats to biodiversity, and impacts on agriculture and food security.

Q: What solutions or strategies are proposed to address climate change?
A: Some of the solutions or strategies proposed to address climate

This code snippet demonstrates how prompt chaining can be used for complex tasks like document analysis and question answering. It breaks down a large task (understanding a long document) into smaller, manageable steps (summarizing sections). It uses the output of one step (summaries) as input for the next (answering questions). It allows for handling documents that might be too long to process in a single prompt.


### Text Generation with Fact Verification
Text generation with fact verification involves generating text and incorporating fact-checking steps within the prompt chain. This ensures the output is not only coherent but also accurate.


In [50]:
def generate_text(topic):
    prompt = f"Write a short paragraph about {topic}."
    return get_completion(prompt)

def extract_facts(text):
    prompt = f"Extract the key factual claims from the following text, listing each claim on a new line:\n\n{text}"
    return get_completion(prompt)

def verify_facts(facts):
    verified_facts = []
    for fact in facts.split('\n'):
        if fact.strip():
            prompt = f"Verify the following statement and respond with 'True' if it's factually correct, 'False' if it's incorrect, or 'Uncertain' if it can't be verified without additional research: '{fact}'"
            verification = get_completion(prompt)
            verified_facts.append((fact, verification.strip()))
    return verified_facts

def revise_text(original_text, verified_facts):
    context = "Original text:\n" + original_text + "\n\nVerified facts:\n"
    for fact, verification in verified_facts:
        context += f"- {fact}: {verification}\n"
    
    prompt = f"{context}\n\nRewrite the original text, keeping the verified facts, removing or correcting any false information, and indicating any uncertain claims as 'It is claimed that...' or similar phrasing."
    return get_completion(prompt)

def text_generation_with_verification(topic):
    print(f"Generating text about: {topic}")
    
    # Step 1: Generate initial text
    initial_text = generate_text(topic)
    print("\nInitial Text:")
    print(initial_text)

    # Step 2: Extract facts
    extracted_facts = extract_facts(initial_text)
    print("\nExtracted Facts:")
    print(extracted_facts)

    # Step 3: Verify facts
    verified_facts = verify_facts(extracted_facts)
    print("\nVerified Facts:")
    for fact, verification in verified_facts:
        print(f"- {fact}: {verification}")

    # Step 4: Revise text
    revised_text = revise_text(initial_text, verified_facts)
    print("\nRevised Text:")
    print(revised_text)

    return revised_text

# Example usage
topic = "the effects of climate change on polar bears"
final_text = text_generation_with_verification(topic)

Generating text about: the effects of climate change on polar bears

Initial Text:
Climate change has had a significant impact on polar bears, as the warming temperatures are causing the sea ice in the Arctic to melt at an alarming rate. This loss of sea ice makes it more difficult for polar bears to hunt for their main food source, seals, leading to decreased body condition and reproductive success. Additionally, as the ice continues to disappear, polar bears are forced to travel longer distances to find food, putting them at risk of exhaustion and starvation. Climate change is also affecting the availability of suitable denning sites for polar bear mothers, further threatening the survival of this iconic species.

Extracted Facts:
- Climate change is causing the sea ice in the Arctic to melt at an alarming rate.
- The loss of sea ice is making it more difficult for polar bears to hunt for seals, their main food source.
- Decreased body condition and reproductive success are being obs

In this code, the `generate_text()` function generates an initial paragraph about the given topic. The `extract_facts()` function extracts key factual claims from the generated text. The `verify_facts()` function takes the extracted facts and verifies each one, classifying them as True, False, or Uncertain. The `revise_text()` function rewrites the original text based on the verified facts, correcting false information and indicating uncertain claims. The `text_generation_with_verification()` function orchestrates the entire process: it generates initial text about the topic, extracts facts from this text, verifies these facts, and finally revises the text based on the verified facts.

The prompt chaining occurs in several steps: the initial text generation, fact extraction from the generated text, verification of each extracted fact, and revision of the text based on verified facts.

This approach demonstrates how prompt chaining can be used for complex tasks like text generation with fact verification. It breaks down the task into smaller, manageable steps (generate, extract, verify, revise). Each step uses the output of the previous step as input, allowing for a systematic approach to ensuring the accuracy of generated content.


### Code Generation with Debugging
Generating code and using subsequent prompts to test and debug it can streamline the development process. This approach ensures the code not only compiles but also performs as expected.


In [51]:
def generate_code(task):
    prompt = f"Write a Python function to {task}. Include comments explaining the code."
    return get_completion(prompt)

def generate_test_cases(code):
    prompt = f"Given the following Python code, generate 3 test cases to verify its functionality. Include both input and expected output for each test case:\n\n{code}"
    return get_completion(prompt)

def run_tests(code, test_cases):
    prompt = f"""
    Given the following Python code and test cases, run the tests and report the results. 
    If any tests fail, explain why and suggest fixes.

    Code:
    {code}

    Test Cases:
    {test_cases}

    For each test case, respond with:
    1. "PASS" if the test passes
    2. "FAIL" if the test fails, along with an explanation of why it failed and a suggested fix
    """
    return get_completion(prompt)

def debug_code(code, test_results):
    prompt = f"""
    Given the following Python code and test results, debug the code to fix any issues.
    Provide the corrected code along with explanations of the changes made.

    Original Code:
    {code}

    Test Results:
    {test_results}
    """
    return get_completion(prompt)

def code_generation_with_debugging(task):
    print(f"Generating code for task: {task}")
    
    # Step 1: Generate initial code
    initial_code = generate_code(task)
    print("\nInitial Code:")
    print(initial_code)

    # Step 2: Generate test cases
    test_cases = generate_test_cases(initial_code)
    print("\nGenerated Test Cases:")
    print(test_cases)

    # Step 3: Run tests
    test_results = run_tests(initial_code, test_cases)
    print("\nTest Results:")
    print(test_results)

    # Step 4: Debug code if necessary
    if "FAIL" in test_results:
        print("\nDebugging code...")
        debugged_code = debug_code(initial_code, test_results)
        print("\nDebugged Code:")
        print(debugged_code)
        
        # Optionally, you can run tests again on the debugged code
        print("\nRe-running tests on debugged code...")
        final_test_results = run_tests(debugged_code, test_cases)
        print("\nFinal Test Results:")
        print(final_test_results)
        
        return debugged_code
    else:
        print("\nAll tests passed. No debugging necessary.")
        return initial_code

# Example usage
task = "calculate the factorial of a number"
final_code = code_generation_with_debugging(task)

Generating code for task: calculate the factorial of a number

Initial Code:
Here is a Python function to calculate the factorial of a number:

```python
def factorial(n):
    # Base case: factorial of 0 is 1
    if n == 0:
        return 1
    # Recursive case: calculate factorial using recursion
    else:
        return n * factorial(n - 1)

# Test the function with an example
number = 5
result = factorial(number)
print(f"The factorial of {number} is {result}")
```

In this code:
- The `factorial` function takes an integer `n` as input and calculates the factorial of `n`.
- The base case checks if `n` is 0 and returns 1, as the factorial of 0 is defined as 1.
- In the recursive case, the function calls itself with `n-1` and multiplies the result by `n` to calculate the factorial.
- The function is then tested with an example number (5 in this case) and the result is printed.

Generated Test Cases:
Test Case 1:
- Input: `number = 0`
- Expected Output: The factorial of 0 is 1

Test Cas

The `generate_code()` function generates initial Python code for the given task. The `generate_test_cases()` function creates test cases for the generated code. The `run_tests()` function simulates running the test cases and reports the results. If any tests fail, the `debug_code()` function attempts to debug and fix the code. The `code_generation_with_debugging()` function orchestrates the entire process: it generates initial code for the task, creates test cases for the code, runs the tests and checks the results, and if any tests fail, it debugs the code and optionally re-runs the tests.

The prompt chaining occurs in several steps: initial code generation based on the task description, test case generation based on the initial code, test execution and result reporting based on the code and test cases, and if necessary, debugging based on the original code and test results.

This demonstrates how prompt chaining can be used for complex tasks like code generation and debugging. It breaks down the software development process into smaller, manageable steps (generate, test, debug). Each step uses the output of the previous step as input, allowing for an iterative approach to code development and improvement.


### Multi-Step Reasoning Tasks
Prompt chaining can be used to solve complex problems requiring multiple reasoning steps, such as mathematical word problems or logical puzzles. Each step builds on the previous one, ensuring a structured and thorough approach.


In [53]:
def solve_step_by_step(problem, max_steps=5):
    steps = []
    current_problem = problem
    for step_num in range(1, max_steps + 1):
        prompt = f"""
        Solve the following problem step by step. Provide step {step_num} only and explain it clearly.
        If this step completes the solution, explicitly state 'SOLUTION COMPLETE' at the end.
        
        Problem: {current_problem}
        """
        step = get_completion(prompt)
        if step is None:
            break
        steps.append(step)
        print(f"Step {step_num}: {step}\n")
        if "SOLUTION COMPLETE" in step:
            break
        current_problem = f"{current_problem}\n\nStep {step_num}: {step}"
    return steps

def combine_steps(steps):
    combined_steps = "\n".join(steps)
    prompt = f"Summarize the following solution steps into a concise final answer:\n\n{combined_steps}"
    return get_completion(prompt)

def multi_step_reasoning(problem):
    print(f"Solving problem: {problem}\n")
    
    # Step 1: Solve step by step
    steps = solve_step_by_step(problem)
    
    # Step 2: Combine steps into a final solution
    if steps:
        final_solution = combine_steps(steps)
        print("Final Solution:")
        print(final_solution)
        return final_solution
    else:
        print("Unable to solve the problem.")
        return None

# Example usage
problem = "A car travels 60 miles per hour for 2 hours, then 40 miles per hour for 3 hours. What is the total distance traveled by the car?"
final_solution = multi_step_reasoning(problem)

Solving problem: A car travels 60 miles per hour for 2 hours, then 40 miles per hour for 3 hours. What is the total distance traveled by the car?

Step 1: Step 1: Calculate the distance traveled at 60 miles per hour for 2 hours.
Explanation: To find the distance traveled at 60 miles per hour for 2 hours, we use the formula: distance = speed × time. 
Distance = 60 miles/hour × 2 hours = 120 miles.

SOLUTION COMPLETE

Final Solution:
The distance traveled at 60 miles per hour for 2 hours is 120 miles.


The `get_completion()` function sends a prompt to the OpenAI API and returns the generated response. The `solve_step_by_step()` function breaks down the problem into smaller steps by repeatedly prompting the model to provide the next step in solving the problem. It stops when the model indicates that the solution is complete or if no further steps are provided, and each step is appended to a list of steps. The `combine_steps()` function combines all the individual steps into a coherent final solution by prompting the model to merge the steps into a single, comprehensive explanation. The `multi_step_reasoning()` function orchestrates the entire process: it starts by solving the problem step by step, then combines the steps into a final solution.

The example problem is: "A car travels 60 miles per hour for 2 hours, then 40 miles per hour for 3 hours. What is the total distance traveled by the car?"

The problem is provided to the model to start the step-by-step solution process. The model is prompted to provide the next step in solving the problem, and each step is collected and printed. Once all steps are collected, they are combined into a final, coherent solution.

